<a href="https://colab.research.google.com/github/Nandinisiddeshuni/CHESS-Datasetsss/blob/main/Populate_Shipping_Database_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
import os

# --- Configuration ---
DATABASE_NAME = 'shipping.db'
SPREADSHEET_DIR = 'forage-walmart-task-4' # Assuming you've cloned the repo into this folder

# --- Database Schema (Assumed based on requirements and common ERD practices) ---
# The script will create these tables if they don't exist.
SCHEMA_SQL = """
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    manufacturer TEXT NOT NULL,
    type TEXT NOT NULL, -- e.g., 'food', 'toy', 'apparel'
    weight REAL,
    flavor TEXT,
    health_condition TEXT,
    material TEXT,
    durability TEXT,
    color TEXT,
    size TEXT,
    care_instructions TEXT,
    UNIQUE(name, manufacturer, type) -- Ensures no duplicate product entries
);

CREATE TABLE IF NOT EXISTS shipments (
    shipment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    shipping_identifier TEXT UNIQUE NOT NULL,
    origin TEXT NOT NULL,
    destination TEXT NOT NULL,
    origin_zip TEXT NOT NULL,
    destination_zip TEXT NOT NULL,
    shipment_date TEXT NOT NULL -- Storing as TEXT for simplicity, could be DATE
);

CREATE TABLE IF NOT EXISTS shipment_items (
    shipment_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    shipment_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    FOREIGN KEY (shipment_id) REFERENCES shipments(shipment_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
"""

def setup_database(db_path):
    """Connects to the SQLite database and ensures tables are created."""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.executescript(SCHEMA_SQL)
    conn.commit()
    print(f"Database '{db_path}' schema ensured.")
    return conn

def get_or_insert_product(cursor, product_data):
    """
    Gets product_id if product exists, otherwise inserts and returns new product_id.
    Handles different product types and their specific attributes.
    """
    # Extract common attributes
    name = product_data.get('product_name')
    manufacturer = product_data.get('manufacturer')
    product_type = product_data.get('product_type')

    # Try to find existing product
    cursor.execute(
        "SELECT product_id FROM products WHERE name = ? AND manufacturer = ? AND type = ?",
        (name, manufacturer, product_type)
    )
    product_id = cursor.fetchone()
    if product_id:
        return product_id[0]

    # If not found, prepare data for insertion. Handle None for nullable columns.
    weight = product_data.get('weight')
    flavor = product_data.get('flavor')
    health_condition = product_data.get('target_health_condition')
    material = product_data.get('material')
    durability = product_data.get('durability')
    color = product_data.get('color')
    size = product_data.get('size')
    care_instructions = product_data.get('specific_care_instructions') # Corrected column name

    insert_sql = """
    INSERT INTO products (name, manufacturer, type, weight, flavor, health_condition,
                          material, durability, color, size, care_instructions)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    cursor.execute(insert_sql, (
        name, manufacturer, product_type, weight, flavor, health_condition,
        material, durability, color, size, care_instructions
    ))
    return cursor.lastrowid

def process_spreadsheet0(file_path, conn):
    """Reads spreadsheet0.xlsx and populates the products table."""
    try:
        df = pd.read_excel(file_path)
        print(f"Processing {os.path.basename(file_path)}...")
    except FileNotFoundError:
        print(f"Error: {file_path} not found. Skipping.")
        return

    cursor = conn.cursor()
    products_inserted = 0

    for index, row in df.iterrows():
        # Prepare product data dictionary, handling varying column names
        product_data = {
            'product_name': row.get('product_name'),
            'manufacturer': row.get('manufacturer'),
            'product_type': row.get('product_type'),
            'weight': row.get('weight'),
            'flavor': row.get('flavor'),
            'target_health_condition': row.get('target_health_condition'),
            'material': row.get('material'),
            'durability': row.get('durability'),
            'color': row.get('color'),
            'size': row.get('size'),
            'specific_care_instructions': row.get('specific_care_instructions') # Matches spreadsheet column
        }

        # Ensure common required fields are not None
        if product_data['product_name'] is None or product_data['manufacturer'] is None or product_data['product_type'] is None:
            print(f"Skipping row {index} in {file_path} due to missing required product data: {product_data}")
            continue

        try:
            get_or_insert_product(cursor, product_data)
            products_inserted += 1
        except sqlite3.IntegrityError as e:
            # This should ideally not happen if UNIQUE constraint is well-defined
            # and get_or_insert_product logic is correct for existing products.
            # However, if new products from spreadsheet0 somehow conflict on other properties, this catches it.
            print(f"Integrity Error inserting product from row {index}: {row['product_name']} - {e}")
        except Exception as e:
            print(f"Error processing product row {index} in {file_path}: {e}")

    conn.commit()
    print(f"Finished processing {os.path.basename(file_path)}. {products_inserted} unique products processed.")

def process_shipment_data(spreadsheet1_path, spreadsheet2_path, conn):
    """
    Reads spreadsheet1.xlsx and spreadsheet2.xlsx, merges them,
    and populates the shipments and shipment_items tables.
    """
    try:
        df1 = pd.read_excel(spreadsheet1_path)
        df2 = pd.read_excel(spreadsheet2_path)
        print(f"Processing {os.path.basename(spreadsheet1_path)} and {os.path.basename(spreadsheet2_path)}...")
    except FileNotFoundError as e:
        print(f"Error: One or both shipment spreadsheets not found ({e}). Skipping.")
        return

    # Merge the two dataframes on 'shipping_identifier'
    # Use 'inner' merge to ensure we only process shipments with complete data
    merged_df = pd.merge(df1, df2, on='shipping_identifier', how='inner')
    print(f"Merged dataframe contains {len(merged_df)} rows for shipment processing.")

    cursor = conn.cursor()
    shipments_processed = 0
    items_inserted = 0

    # Track inserted shipments to avoid re-inserting shipment header data
    inserted_shipments = set()

    # Iterate through the merged dataframe
    for index, row in merged_df.iterrows():
        shipping_identifier = row.get('shipping_identifier')

        if shipping_identifier is None:
            print(f"Skipping row {index} due to missing shipping_identifier.")
            continue

        # --- Process Shipment Header (if not already processed) ---
        if shipping_identifier not in inserted_shipments:
            origin = row.get('origin')
            destination = row.get('destination')
            origin_zip = row.get('origin_zip')
            destination_zip = row.get('destination_zip')
            shipment_date = str(row.get('shipment_date')) # Convert date to string for TEXT column

            if any(val is None for val in [origin, destination, origin_zip, destination_zip, shipment_date]):
                print(f"Skipping shipment {shipping_identifier} due to missing header data.")
                continue

            try:
                cursor.execute(
                    """
                    INSERT INTO shipments (shipping_identifier, origin, destination, origin_zip, destination_zip, shipment_date)
                    VALUES (?, ?, ?, ?, ?, ?)
                    ON CONFLICT(shipping_identifier) DO NOTHING
                    """,
                    (shipping_identifier, origin, destination, origin_zip, destination_zip, shipment_date)
                )
                if cursor.rowcount > 0: # Check if a new row was inserted
                    shipments_processed += 1
                inserted_shipments.add(shipping_identifier) # Mark as processed
            except sqlite3.IntegrityError as e:
                # This could happen if ON CONFLICT DO NOTHING fails for some reason
                print(f"Integrity Error inserting shipment {shipping_identifier}: {e}")
            except Exception as e:
                print(f"Error processing shipment header for {shipping_identifier}: {e}")

        # --- Retrieve Shipment ID ---
        cursor.execute("SELECT shipment_id FROM shipments WHERE shipping_identifier = ?", (shipping_identifier,))
        shipment_id_result = cursor.fetchone()
        if not shipment_id_result:
            print(f"Could not retrieve shipment_id for {shipping_identifier}. Skipping shipment item.")
            continue
        shipment_id = shipment_id_result[0]

        # --- Process Shipment Item ---
        product_name = row.get('product_name')
        manufacturer = row.get('manufacturer')
        quantity = row.get('quantity')

        # Ensure product name, manufacturer, and quantity are not None for shipment item
        if any(val is None for val in [product_name, manufacturer, quantity]):
            print(f"Skipping shipment item in row {index} due to missing product/quantity data.")
            continue

        # Get product_id from the products table (assuming it's already populated by spreadsheet0)
        # We need to make an educated guess about product type for lookup
        # For shipment data, product type is not explicitly given, so we need to query
        # the 'products' table more broadly.
        cursor.execute(
            "SELECT product_id FROM products WHERE name = ? AND manufacturer = ?",
            (product_name, manufacturer)
        )
        product_id_result = cursor.fetchone()

        if not product_id_result:
            print(f"Warning: Product '{product_name}' by '{manufacturer}' not found in catalog. Skipping shipment item in row {index}.")
            continue
        product_id = product_id_result[0]

        try:
            cursor.execute(
                """
                INSERT INTO shipment_items (shipment_id, product_id, quantity)
                VALUES (?, ?, ?)
                """,
                (shipment_id, product_id, quantity)
            )
            items_inserted += 1
        except Exception as e:
            print(f"Error inserting shipment item for shipment {shipping_identifier}, product {product_name}: {e}")

    conn.commit()
    print(f"Finished processing shipment data. {shipments_processed} unique shipments processed, {items_inserted} shipment items inserted.")


# --- Main Execution ---
if __name__ == "__main__":
    db_file_path = os.path.join(SPREADSHEET_DIR, DATABASE_NAME)
    spreadsheet0_path = os.path.join(SPREADSHEET_DIR, 'spreadsheet0.xlsx')
    spreadsheet1_path = os.path.join(SPREADSHEET_DIR, 'spreadsheet1.xlsx')
    spreadsheet2_path = os.path.join(SPREADSHEET_DIR, 'spreadsheet2.xlsx')

    # 1. Setup Database
    conn = None
    try:
        conn = setup_database(db_file_path)

        # 2. Process Spreadsheet 0 (Product Catalog)
        process_spreadsheet0(spreadsheet0_path, conn)

        # 3. Process Spreadsheets 1 & 2 (Shipments and Shipment Items)
        process_shipment_data(spreadsheet1_path, spreadsheet2_path, conn)

    except Exception as e:
        print(f"An unexpected error occurred during database population: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

An unexpected error occurred during database population: unable to open database file
